# Example 3: Simulating an Electric Vehicle with an LPV Model

This example follows the previous example, but where we instead assume that each cell in the battery pack are modeled as a Linear Parameter-Varying (LPV) model. The model is assmued to follow the structure:

$$V_q[k] = -\sum_{i=1}^n a_i\big(SOC[k-i], I[k-1]\big) V_q[k-i] + \sum_{i=0}^n b_i\big(SOC[k-i], I[k-i]\big) V_q[k-i]$$
where
$$V_q[k] = V_t[k] - OCV[k]$$
$$a_i\big(SOC[k-i], I[k-i]\big) = \sum_{j=1}^l a_{ij}\phi_j\big(SOC[k-i], I[k-i]\big)$$
$$b_i\big(SOC[k-i], I[k-i]\big) = \sum_{j=1}^l b_{ij}\phi_j\big(SOC[k-i], I[k-i]\big)$$

and $\phi(\cdot)$ represent a nonlinear basis function. The model order and nonlinearity order are represented by $n$ and $l$, respectively.

In [ ]:
from tracksim.tracksim import Vehicle, Pack
from tracksim.vehicle_models import ChevyVoltTuned
from tracksim.pack_models import ChevyVoltPack
from tracksim.cell_models import load_LPV_2_1 # first order ARX with dynamic parameters

cell_model = load_LPV_2_1()

print(cell_model)

Since the nominal capacity of this cell is relatively small, we can increase the dimensions of the battery pack to raise the nominal energy capacity.

In [ ]:
pack_model = ChevyVoltPack.copy()

pack_model['Number of cells in series'] = 36
pack_model['Number of cells in parallel'] = 16

The setup and simulation of the vehicle is the same as before.

In [ ]:
pack = Pack(pack_model, cell_model)
vehicle = Vehicle(ChevyVoltTuned, pack)

In [ ]:
import pandas as pd
from tracksim.utils import exp_average

# Get trip data

trip_file = 'simulated_trip_files/veh0.csv'
trip_data = pd.read_csv(trip_file)

time = trip_data['Time [s]']
time_delta = time[1] - time[0]
speed = exp_average(trip_data['Speed [m/s]'], 0.2) # Low-pass filter to make a more realistic speed profile

# Set initial battery conditions

vehicle.simulate_vehicle(time, speed, time_delta)
vehicle.simulate_battery_pack()

We can then retrieve the same vehicle, pack, and cell measurements as in the previous example.

In [ ]:
from tracksim.utils import plot_vehicle_and_battery_data

fig, ax = plot_vehicle_and_battery_data(vehicle)

We can also extract the varying LPV parameters. Since the LPV is a 2nd-order model, the available parameters are $a_1$, $a_2$, $b_0$, $b_1$, and $b_2$.

In [ ]:
a1 = vehicle.pack.simulation_results['Cell 0-0']['a1']
a2 = vehicle.pack.simulation_results['Cell 0-0']['a2']
b0 = vehicle.pack.simulation_results['Cell 0-0']['b0']
b1 = vehicle.pack.simulation_results['Cell 0-0']['b1']
b2 = vehicle.pack.simulation_results['Cell 0-0']['b2']

fig, ax = plt.subplots(nrows=5)
ax[0].plot(a1)
ax[0].set_ylabel(r'$a_1$')
ax[1].plot(a2)
ax[1].set_ylabel(r'$a_2$')
ax[2].plot(b0)
ax[2].set_ylabel(r'$b_0$')
ax[3].plot(b1)
ax[3].set_ylabel('$b_1$')
ax[4].plot(b2)
ax[4].set_ylabel('$b_2$')
ax[4].set_xlabel('Time [s]')

fig.tight_layout()